In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
SPARK_HOME = '/home/jose/Frameworks/spark-3.0.2-bin-hadoop2.7'

In [3]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

import findspark
findspark.init(SPARK_HOME)

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

config = SparkConf() \
                    .setMaster('local[*]') \
                    .setAppName('Spark Base') \
                    .setAll([('spark.executor.memory', '2G'),
                            ('spark.driver.memory', '2G'),
                            ('spark.driver.maxResultSize', '1G')])

sc = SparkContext(conf=config)
spark = SparkSession(sc)

spark

## Create Dataframe

### No schema

In [6]:
tuple_list_data = [('Alice', 1), ('Braga', 2), ('Steve', 3)]
spark.createDataFrame(tuple_list_data, ['name', 'id']).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [7]:
dict_list_data = [{'name': 'Alice', 'id': 1}, {'name': 'Braga', 'id': 2}, {'name': 'Steve', 'id': 2}]
spark.createDataFrame(dict_list_data).show()

/home/jose/Frameworks/spark-3.0.2-bin-hadoop2.7/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|Braga|
|  2|Steve|
+---+-----+



In [19]:
from pyspark.sql.types import Row

row_list_data = [Row('Alice', 1), Row('Braga', 2), Row('Steve', 3)]
spark.createDataFrame(row_list_data, ['name', 'id']).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



### With Schema

In [16]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField('name', StringType()),
    StructField('id', IntegerType())
])

spark.createDataFrame(tuple_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [17]:
spark.createDataFrame(dict_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [20]:
spark.createDataFrame(row_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



## Table

In [26]:
sample_data = [('Alice', 1), ('Braga', 2), ('Steve', 3)]
df = spark.createDataFrame(sample_data, ['name', 'id'])
df.createOrReplaceTempView("my_table")

In [31]:
r = spark.sql('SELECT * FROM my_table')
r.show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



# Basic Operations with SQL

In [4]:
sprinters = [
    ('Usain Bolt', 1),
    ('Tyson Gay', 2),
    ('Yohan Blake', 3),
    ('Asafa Powell', 4),
    ('Justin Gatlin', 5),
    ('Christian Coleman', 6)
]

times = {
    1: 9.58,
    2: 9.69,
    3: 9.72,
    4: 9.74,
    5: 9.76,
    6: 9.78
}

sprinters_df = spark.createDataFrame(
    sprinters,
    schema=['name', 'id']
)

times_df = spark.createDataFrame(
    times.items(),
    schema=['id', 'time']
)


# SQL
sprinters_df.createOrReplaceTempView('sprinters')
times_df.createOrReplaceTempView('times')

In [5]:
spark.sql("""
    SELECT *
    FROM sprinters
    """).toPandas()

,name,id
0,Usain Bolt,1
1,Tyson Gay,2
2,Yohan Blake,3
3,Asafa Powell,4
4,Justin Gatlin,5
5,Christian Coleman,6


In [6]:
sprinters_and_times_df = spark.sql("""
    SELECT * 
    FROM sprinters s
    INNER JOIN times t ON s.id = t.id
""")

sprinters_and_times_df.toPandas()

,name,id,id,time
0,Christian Coleman,6,6,9.78
1,Justin Gatlin,5,5,9.76
2,Usain Bolt,1,1,9.58
3,Yohan Blake,3,3,9.72
4,Tyson Gay,2,2,9.69
5,Asafa Powell,4,4,9.74


## Window

In [59]:
spark.sql("""
    SELECT * 
    FROM sprinters s
    INNER JOIN times t ON s.id = t.id
    ORDER BY time
""").toPandas()

,name,id,id,time
0,Usain Bolt,1,1,9.58
1,Tyson Gay,2,2,9.69
2,Yohan Blake,3,3,9.72
3,Asafa Powell,4,4,9.74
4,Justin Gatlin,5,5,9.76
5,Christian Coleman,6,6,9.78


In [60]:
from pyspark.sql import Window
from pyspark.sql.functions import rank


spark.sql("""
    SELECT name
    FROM (
        SELECT *, RANK() OVER (ORDER BY time) AS rank 
        FROM sprinters s
        INNER JOIN times t ON s.id = t.id
    ) as t
    WHERE t.rank = 1
""").toPandas()


window = Window().orderBy('time')

sprinters_df.join(times_df, on="id") \
            .withColumn("rank", rank().over(window)) \
            .filter("rank == 1") \
            .toPandas()

,id,name,time,rank
0,1,Usain Bolt,9.58,1


## CASE Clause

In [61]:
from pyspark.sql.functions import when, col

spark.sql("""
    SELECT name,
    CASE
        WHEN t.time <= 9.58 THEN "Bolt"
        WHEN t.time > 9.58 AND t.time <= 9.7 THEN "Almost a Bolt"
        ELSE "Not Bolt"
    END AS comment
    FROM sprinters s
    JOIN times t ON s.id = t.id
""").toPandas()


sprinters_df.join(times_df, on="id") \
            .withColumn("comment", when(col("time") <= 9.58, "Bolt").when((col("time") > 9.58) & (col("time") <= 9.7), "Almost a Bolt").otherwise("Not Bolt")) \
            .toPandas()

,id,name,time,comment
0,6,Christian Coleman,9.78,Not Bolt
1,5,Justin Gatlin,9.76,Not Bolt
2,1,Usain Bolt,9.58,Bolt
3,3,Yohan Blake,9.72,Not Bolt
4,2,Tyson Gay,9.69,Almost a Bolt
5,4,Asafa Powell,9.74,Not Bolt


In [62]:
spark.sql("""
    SELECT name, time
    FROM sprinters s
    JOIN times t ON s.id = t.id
    ORDER BY CASE
              WHEN t.time <= 9.58 THEN "Bolt"
              WHEN t.time > 9.58 AND t.time <= 9.7 THEN "Almost a Bolt"
              ELSE "Not Bolt"
          END
""").toPandas()

,name,time
0,Tyson Gay,9.69
1,Usain Bolt,9.58
2,Christian Coleman,9.78
3,Justin Gatlin,9.76
4,Yohan Blake,9.72
5,Asafa Powell,9.74


## Aggregation

In [64]:
dealer = [
    (100, 'Fremont', 'Honda Civic', 10),
    (100, 'Fremont', 'Honda Accord', 15),
    (100, 'Fremont', 'Honda CRV', 7),
    (200, 'Dublin', 'Honda Civic', 20),
    (200, 'Dublin', 'Honda Accord', 10),
    (200, 'Dublin', 'Honda CRV', 3),
    (300, 'San Jose', 'Honda Civic', 5),
    (300, 'San Jose', 'Honda Accord', 8)
]

dealer_df = spark.createDataFrame(
    dealer,
    schema=["id", "city", "car_model", "quantity"]
)

dealer_df.createOrReplaceTempView("dealer")

In [84]:
from pyspark.sql.functions import sum as pysum

spark.sql("""
    SELECT id, sum(quantity)
    FROM dealer
    GROUP BY id
    ORDER BY id
""").toPandas()



dealer_df.groupby("id") \
         .agg({"quantity": "sum"}) \
         .sort("id") \
         .toPandas()

dealer_df.groupby("id") \
         .sum("quantity") \
         .sort("id") \
         .toPandas()

dealer_df.groupby("id") \
         .agg(pysum("quantity").alias("sum")) \
         .sort("id") \
         .toPandas()

,id,sum
0,100,32
1,200,33
2,300,13


In [88]:
spark.sql("""
    SELECT car_model, count(car_model)
    FROM dealer
    GROUP BY car_model
""").toPandas()

# dealer_df.groupby("car_model") \
#          .count() \
#          .toPandas()

,car_model,count(car_model)
0,Honda Civic,3
1,Honda CRV,2
2,Honda Accord,3
